### Decripcion del agente

En esta ocasión desarrollaremos un agente que ayude en la gestión efectiva del tiempo       
el mismo, medirá la “productividad” del usuario. Dicha “productividad” corresponde       
a la cantidad de tareas realizadas en un tiempo determinado.       
A su vez, el agente le recomendara al usuario acciones a tomar para un mayor provecho del tiempo.

*Librerias a usar para la contruccion del agente*

Como libería, pensamos utilizar MESA de python puesto que en la descripción de la librería       
nos indica que está orientado a horarios de agentes, por lo cual podríamos basarnos       
en algunos de los métodos que ya la librería implementa,        
sin embargo, es posible que también utilicemos otras liberías como AgentPy       
puesto que en Mesa no estará todo lo que necesitamos 

### Implementacion del agente

In [5]:
import numpy
import mesa
import agentpy
import ipysimulate
import ipywidgets
import pandas as pd
from anytree import NodeMixin, RenderTree

Estas son las importaciones de las librerías orientadas a IA que utilizaremos:
mesa, agentpy, ipysimulate
Usaremos además otras librerías auxiliares, cómo:
numpy, pandas, anytree


In [6]:
class Task:
    def __init__(self, id, name, avgTime, urgency):
        self.id=id
        self.name=name
        self.avgTime=avgTime
        self.urgency=urgency
    
    def __str__(self) -> str:
        """
        To String del objeto Task. 
        Llevar los atributos del objeto a una forma legible a la
        hora de imprimir el objeto.

        Args:
            self (Task): Instancia de la clase Task.
        
        Returns:
            str: Representacion como texto del objeto Task.
        """
        return f"Name: {self.name}, AvgTime: {self.avgTime}, Urgency: {self.urgency}"



class TaskClass(Task, NodeMixin):  # Add Node feature
    def __init__(self, name, avgTime, urgency, parent=None, children=None):
        super(Task, self).__init__()
        self.name = name
        self.avgTime = avgTime
        self.urgency = urgency
        self.parent = parent
        if children:
            self.children = children

In [1]:
class Choice():
    def __init__(self, move, urgency):
        self.move = move
        self.urgency = urgency



    def __str__(self):
        """
        To String del objeto Choice. 
        Llevar los atributos del objeto a una forma legible a la
        hora de imprimir el objeto.

        Args:
            self (Choice): Instancia de la clase Choice.
        
        Returns:
            str: Representacion como texto del objeto Choice.
        """
        return self.move + ": " + str(self.urgency)
    
    
    

In [18]:
#definamos la estructura más básica de nuestro agente
#la cual podrá ir creciendo según se definan nuevas características

class ScheduleAgent:
  def __init__(self, name:str, age:int, gender:str, taskList:pd.DataFrame, comments):
    self.name = name
    self.age = age
    self.gender = gender
    self.taskList = taskList
    self.notification = comments
    self.comments = comments
    self.rootNode = self.buildTree() #creo el arbol al inicializar el objeto
    
    
  
  def getNodesList(self) -> list:
    """
    Crear una lista de nodos del arbol de tareas de acuerdo a las
    tareas almacenadas en un archivo CSV.

    Args:
        self (ScheduleAgent): Instancia de la clase ScheduleAgent.

    Returns:
        list: Lista de nodos del arbol de tareas, cada nodo representa una tarea.
    """
    nodeList = []
    for i in range(0, len(self.taskList["Name"])):
        nodeList.append(TaskClass(
            name = self.taskList["Name"][i],
            avgTime = self.taskList["AvgTime"][i],
            urgency = self.taskList["Urgency"][i]
        ))
    return nodeList
  
  
  
  def setParentNodes(self, nodeList:list) -> TaskClass:
    """
    Colocar a cada uno de los nodos del arbol de tareas su respectivo nodo padre.

    Args:
        self (ScheduleAgent): Instancia de la clase ScheduleAgent.
        nodeList (list): Lista de nodos del arbol de tareas, cada nodo representa una tarea.

    Returns:
        TaskClass: Nodo raiz del arbol de tareas, desde el nodo raiz se puede recorrer la totalidad del arbol.
    """
    for i in range(1, len(nodeList)):
        node = nodeList[i]
        node.parent = nodeList[(int(self.taskList["Parent"][i]))-1]
    return nodeList[0] #retorno root del arbol ya que desde root lo puedo recorrer todo

  
  
  def buildTree(self) -> TaskClass: 
    """
    Armar el arbol de tareas.
    En primera instancia crea una lista de nodos de tareas para 
    despues asignarles su respectivo nodo padre.

    Args:
        self (ScheduleAgent): Instancia de la clase ScheduleAgent.

    Returns:
        TaskClass: Nodo raiz del arbol de tareas, desde el nodo raiz se puede recorrer la totalidad del arbol.
    """
    nodeList = self.getNodesList()
    rootNode = self.setParentNodes(nodeList)
    return rootNode #retorno root del arbol ya que desde root lo puedo recorrer todo
    
    
    
  def printTaskTree(self) -> None:
    """
    Imprimir por completo el arbol de tareas. 
    
    Args:
        self (ScheduleAgent): Instancia de la clase ScheduleAgent.
    """
    for pre, fill, node in RenderTree(self.rootNode):
        print("%s%s" % (pre, node.name))   
  
  
  
  def miniMax(self, rootNode:Task, isMax:bool) -> Choice:
    """
    Algoritmo de busqueda miniMax para determinar la mayor cantidad de tareas de la mayor urgencia posible
    a completar en el minimo tiempo posible.

    Args:
        self (ScheduleAgent): Instancia de la clase ScheduleAgent.
        rootNode (Task): Nodo raiz del arbol de tareas, desde el nodo raiz se puede recorrer la totalidad del arbol.
        isMax (bool): ?

    Returns:
        Choice: Eleccion de rama y nodo del algortimo miniMax
    """
    children = rootNode.children
    try:
        l_choice = self.miniMax(children[0], not isMax)
        r_choice = self.miniMax(children[1], not isMax)

        if (isMax):
            if (l_choice.urgency > r_choice.urgency):
                return Choice("left", l_choice.urgency)
            else:
                return Choice("right", r_choice.urgency)
        else:
            if (l_choice.urgency < r_choice.urgency):
                return Choice("left", l_choice.urgency)
            else:
                return Choice("right", r_choice.urgency) 
    except IndexError:
        return Choice("end", self.rootNode.urgency)
      
   
      

In [24]:
#inicializando el agente para el usuario Ricardo
myAgent = ScheduleAgent(name = "Ricardo", 
                        age = 36, 
                        gender = "Male", 
                        taskList = pd.read_csv("./taskList.csv"),
                        comments = "")

isMax = True
currentNode = myAgent.rootNode

while (True):
    # run minimax on current node
    agentChoice = myAgent.miniMax(currentNode, isMax) 
    
    # make choice based on minimax search
    if (agentChoice.move == "left"):
        print ("Nos movemos a la izquierda con una urgencia de: " + str(currentNode.children[0].urgency))
        currentNode = currentNode.children[0]
    elif (agentChoice.move == "right"):
        print ("Nos movemos a la derecha con una urgencia de: " + str(currentNode.children[1].urgency))
        currentNode = currentNode.children[1]
    elif (agentChoice.move == "end"):
        print ("La urgencia del último nodo hoja es: " + str(agentChoice.urgency))
        break

Nos movemos a la derecha con una urgencia de: 5
Nos movemos a la derecha con una urgencia de: 7
Nos movemos a la derecha con una urgencia de: 6
La urgencia del último nodo hoja es: 5
